# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [1]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [2]:
df = pd.read_excel(r'C:\Users\user\аварии\Экс4.xlsx')
df.head()

,Тип нарушения,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,Документация,1.000000,1.000000,8.000000,2.000000,9.0,8.0,7
1,Экспертиза промышленной безопасности,1.000000,1.000000,7.000000,8.000000,5.0,1.0,7
2,Готовность к ликвидации последствий аварий и и...,0.125000,0.142857,1.000000,1.000000,7.0,1.0,7
3,Оформление информационными знаками и знаками б...,0.500000,0.125000,1.000000,1.000000,9.0,1.0,8
4,Содержание объектов и оборудования,0.111111,0.200000,0.142857,0.111111,1.0,9.0,7


In [3]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (7, 8)


In [4]:
# типы столбцов 
df.dtypes

Тип нарушения                                                 object
Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [5]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Тип нарушения') 
print(df)

   Документация   Экспертиза промышленной безопасности  \
0       1.000000                              1.000000   
1       1.000000                              1.000000   
2       0.125000                              0.142857   
3       0.500000                              0.125000   
4       0.111111                              0.200000   
5       0.125000                              1.000000   
6       0.142857                              0.142857   

   Готовность к ликвидации последствий аварий и инцидентов  \
0                                           8.000000         
1                                           7.000000         
2                                           1.000000         
3                                           1.000000         
4                                           0.142857         
5                                           1.000000         
6                                           0.142857         

   Оформление информационными знаками 

In [6]:
# типы столбцов 
df.dtypes

Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [7]:
# также меням тип данных Документации
df['Документация '] = df['Документация '].astype(float)

In [8]:
df.loc[:, df.columns[df.dtypes == 'float64']].head(7)

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ)
0,1.000000,1.000000,8.000000,2.000000,9.000000,8.000000
1,1.000000,1.000000,7.000000,8.000000,5.000000,1.000000
2,0.125000,0.142857,1.000000,1.000000,7.000000,1.000000
3,0.500000,0.125000,1.000000,1.000000,9.000000,1.000000
4,0.111111,0.200000,0.142857,0.111111,1.000000,9.000000
5,0.125000,1.000000,1.000000,1.000000,0.111111,1.000000
6,0.142857,0.142857,0.142857,0.125000,0.142857,0.142857


In [9]:
DF_1=df[:].copy()

In [10]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Документация                                                  3.003968
Экспертиза промышленной безопасности                          3.610714
Готовность к ликвидации последствий аварий и инцидентов      18.285714
Оформление информационными знаками и знаками безопасности    13.236111
Содержание объектов и оборудования                           31.253968
Охранные зоны (ОЗ)                                           21.142857
Организация рабочего процесса и безопасности персонала       44.000000
dtype: float64

In [11]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Документация'] = df['Документация '].astype(float).div(df['Документация '].sum())
df['new_Экспертиза'] = df['Экспертиза промышленной безопасности'].astype(float).div(df['Экспертиза промышленной безопасности'].sum())
df['new_Готовность'] = df['Готовность к ликвидации последствий аварий и инцидентов'].astype(float).div(df['Готовность к ликвидации последствий аварий и инцидентов'].sum())
df['new_Оформление'] = df['Оформление информационными знаками и знаками безопасности'].astype(float).div(df['Оформление информационными знаками и знаками безопасности'].sum())
df['new_Содержание'] = df['Содержание объектов и оборудования '].astype(float).div(df['Содержание объектов и оборудования '].sum())
df['new_ОЗ'] = df['Охранные зоны (ОЗ)'].astype(float).div(df['Охранные зоны (ОЗ)'].sum())
df['new_Организация'] = df['Организация рабочего процесса и безопасности персонала'].astype(float).div(df['Организация рабочего процесса и безопасности персонала'].sum())

In [12]:
df.head()

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,1.000000,1.000000,8.000000,2.000000,9.0,8.0,7,0.332893,0.276954,0.437500,0.151102,0.287963,0.378378,0.159091
1,1.000000,1.000000,7.000000,8.000000,5.0,1.0,7,0.332893,0.276954,0.382812,0.604407,0.159980,0.047297,0.159091
2,0.125000,0.142857,1.000000,1.000000,7.0,1.0,7,0.041612,0.039565,0.054687,0.075551,0.223972,0.047297,0.159091
3,0.500000,0.125000,1.000000,1.000000,9.0,1.0,8,0.166446,0.034619,0.054687,0.075551,0.287963,0.047297,0.181818
4,0.111111,0.200000,0.142857,0.111111,1.0,9.0,7,0.036988,0.055391,0.007813,0.008395,0.031996,0.425676,0.159091


In [13]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Документация', 'new_Экспертиза', 'new_Готовность', 'new_Оформление', 'new_Содержание', 'new_ОЗ', 'new_Организация']]
df_new

,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,0.332893,0.276954,0.437500,0.151102,0.287963,0.378378,0.159091
1,0.332893,0.276954,0.382812,0.604407,0.159980,0.047297,0.159091
2,0.041612,0.039565,0.054687,0.075551,0.223972,0.047297,0.159091
3,0.166446,0.034619,0.054687,0.075551,0.287963,0.047297,0.181818
4,0.036988,0.055391,0.007813,0.008395,0.031996,0.425676,0.159091
5,0.041612,0.276954,0.054687,0.075551,0.003555,0.047297,0.159091
6,0.047556,0.039565,0.007813,0.009444,0.004571,0.006757,0.022727


In [14]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация,mean
0,0.332893,0.276954,0.437500,0.151102,0.287963,0.378378,0.159091,0.289126
1,0.332893,0.276954,0.382812,0.604407,0.159980,0.047297,0.159091,0.280491
2,0.041612,0.039565,0.054687,0.075551,0.223972,0.047297,0.159091,0.091682
3,0.166446,0.034619,0.054687,0.075551,0.287963,0.047297,0.181818,0.121198
4,0.036988,0.055391,0.007813,0.008395,0.031996,0.425676,0.159091,0.103621


In [15]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0

In [16]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new8 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

df_new4 =DF_1.loc[3]

df_new5 =DF_1.loc[4]

df_new6 =DF_1.loc[5]

df_new7 =DF_1.loc[6]

In [17]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new8)
print(res1)
res2 = np.dot(df_new2,df_new8)
print(res2)
res3 = np.dot(df_new3,df_new8)
print(res3)
res4 = np.dot(df_new4,df_new8)
print(res4)
res5 = np.dot(df_new5,df_new8)
print(res5)
res6 = np.dot(df_new6,df_new8)
print(res6)
res7 = np.dot(df_new7,df_new8)
print(res7)

3.36934469916746
2.9316164135947975
1.2469777055610136
1.5774095604723246
1.2038006349082964
0.7735632876180402
0.15764377851700437


In [18]:
sum_new = res1 + res2 + res3 + res4 + res5 + res6 + res7
print(sum_new)

11.260356079838935


In [19]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 7)/(7-1)
print(CI)
RI = (1.98*(7-2))/7
print(RI)
CR = CI/RI
print(CR)

0.7100593466398225
1.4142857142857144
0.5020621642907835


In [20]:
# Получилось, что значение CR=0.5, что больше 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами